## Skateboard Problem

In [1]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [2]:
#Create LP model object
mymodel = glp.Solver('Skateboard',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [3]:
inf = mymodel.infinity()

         # name: supply capacity, objective coefficient list
source = {'D': (350 , [25,25,35,40]),    # Detroit capacity and shipping cost per unit
          'LA': (350 , [35,45,35,42.50]),  # Los Angles capacity and shipping cost per unit 
          'A': (700 , [40,40,42.50,32.50])}  # Austin capacity and shipping cost per unit 

distribute = {'I': [30,27.50,30],    # Iowa shipping cost per unit to each Retailer
             'M': [20,32.50,40],    # Maryland shipping cost per unit to each Retailer
             'Id':  [35,40,32.50],   # Idaho shipping cost per unit to each Retailer
             'A':  [27.50,25,42.50]}    #  Arkansas shipping cost per unit to each Reailer
             

        # name: demand quantity required
demand = {'JS':200 , 'SS':500 , 'SD':650}    # weekly customer demand for Just Sports, Sports n Stuff, and Sports Dude

mymodel.Objective().SetMinimization()

In [4]:
# create inbound shipment variables from suppliers to Distribution Center and a dictionary to access them
ship = dict()
for s in source:
    ship[s] = dict()
    (b,coeff_lst ) = source[s]
    for (t,c) in zip(distribute,coeff_lst):
        ship[s][t] = mymodel.NumVar(0,inf, s +'.' + t)
        mymodel.Objective().SetCoefficient(ship[s][t], c )

In [5]:
# create outbound shipment variables from Distribution Center to Retailers  and a dictionary to access them
for t in distribute:
    ship[t] = dict()
    coeff_lst = distribute[t]
    for (d,c) in zip(demand,coeff_lst):
        ship[t][d] = mymodel.NumVar(0,inf, t +'.' + d )
        mymodel.Objective().SetCoefficient(ship[t][d], c)

In [6]:
# create source/supply constraints
for s in source:
    (b,coeff_lst) = source[s]
    constr = mymodel.Constraint(-inf,b,s )
    for t in distribute:
        constr.SetCoefficient(ship[s][t], 1)

KeyError: 'I'

In [7]:
# create demand constraints
for d in demand:
    lb = ub = demand[d]
    constr = mymodel.Constraint(lb,ub,d)
    for t in distribute:
        constr.SetCoefficient(ship[t][d], 1 )

In [8]:
# create transshipment flow balance constraints
for t in distribute:
    constr = mymodel.Constraint(0,0,t )
    for s in source:
        constr.SetCoefficient(ship[s][t],1 )
    for d in demand:
        constr.SetCoefficient(ship[t][d],-1 )

KeyError: 'I'

In [9]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 36000.00
D.I = 0.00
D.M = 0.00
D.Id = 0.00
D.A = 0.00
LA.I = 0.00
LA.M = 0.00
LA.Id = 0.00
LA.A = 0.00
A.I = 0.00
A.M = 0.00
A.Id = 0.00
A.A = 0.00
I.JS = 0.00
I.SS = 0.00
I.SD = 650.00
M.JS = 200.00
M.SS = 0.00
M.SD = 0.00
Id.JS = 0.00
Id.SS = 0.00
Id.SD = 0.00
A.JS = 0.00
A.SS = 500.00
A.SD = 0.00


In [10]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
     D.I    0.0    0.0    inf  25.00
     D.M    0.0    0.0    inf  25.00
    D.Id    0.0    0.0    inf  35.00
     D.A    0.0    0.0    inf  40.00
    LA.I    0.0    0.0    inf  35.00
    LA.M    0.0    0.0    inf  45.00
   LA.Id    0.0    0.0    inf  35.00
    LA.A    0.0    0.0    inf  42.50
     A.I    0.0    0.0    inf  40.00
     A.M    0.0    0.0    inf  40.00
    A.Id    0.0    0.0    inf  42.50
     A.A    0.0    0.0    inf  32.50
    I.JS    0.0    0.0    inf  10.00
    I.SS    0.0    0.0    inf   2.50
    I.SD    0.0  650.0    inf   0.00
    M.JS    0.0  200.0    inf   0.00
    M.SS    0.0    0.0    inf   7.50
    M.SD    0.0    0.0    inf  10.00
   Id.JS    0.0    0.0    inf  15.00
   Id.SS    0.0    0.0    inf  15.00
   Id.SD    0.0    0.0    inf   2.50
    A.JS    0.0    0.0    inf   7.50
    A.SS    0.0  500.0    inf   0.00
    A.SD    0.0    0.0    inf  12.50


In [11]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
         D   -inf    0.0  350.0   0.00
        LA   -inf    0.0  350.0   0.00
         A   -inf    0.0  700.0   0.00
        JS  200.0  200.0  200.0  20.00
        SS  500.0  500.0  500.0  25.00
        SD  650.0  650.0  650.0  30.00
         I    0.0    0.0    0.0   0.00


In [12]:
# They should invest in the expansion of the Iowa DC as the profit outweighs the amortized cost

## Shortest Path Problem

In [13]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [14]:
#Create LP model object
mymodel = glp.Solver('ShortestPath',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [15]:
inf = mymodel.infinity()

# node dictionary; value = -1 is origin, value = 1 is destination, value = 0 is intermediate node
node = {1: -1, 2:0 , 3:0 , 4:0 , 5:0 , 6:0 , 7: 1}

# edge dictionary; (origin,destination):distance/cost
edge = {(1,2):7,
        (1,3):9,
        (1,4):18,
        (2,3):3,
        (3,2):3,
        (2,5):5,
        (5,2):5,
        (3,5):4,
        (5,3):4,
        (4,6):3,
        (5,6):2,
        (6,5):2,
        (5,7):6,
        (6,7):3}

mymodel.Objective().SetMinimization()    # minimize total transportation cost

In [16]:
# create a constraint dictionary containing a constraint for each node
constr = dict()
for n in  node:
    b =  node[n]
    constr[n] = mymodel.Constraint(b,b,str(n) )

In [17]:
# create a variable dictionary containing a variable for each edge
# add each variable to the objective and its corresponding constraints

path = dict()
for e in  edge:
    (o,d ) = e
    c = edge[e]
    path[e] = mymodel.NumVar(0,inf,str(o) + '_' + str(d) )
    mymodel.Objective().SetCoefficient(path[e], c)
    constr[o].SetCoefficient(path[e],-1)
    constr[d].SetCoefficient(path[e], 1 )

In [18]:
lpt.print_model(mymodel)

Variables:
1_2, 1_3, 1_4, 2_3, 3_2, 2_5, 5_2, 3_5, 5_3, 4_6, 5_6, 6_5, 5_7, 6_7 

minimize: 7.0*1_2 + 9.0*1_3 + 18.0*1_4 + 3.0*2_3 + 3.0*3_2 + 5.0*2_5 + 5.0*5_2 + 4.0*3_5 + 4.0*5_3 + 3.0*4_6 + 2.0*5_6 + 2.0*6_5 + 6.0*5_7 + 3.0*6_7 

Subject To:
1: - 1.0*1_2 - 1.0*1_3 - 1.0*1_4 = -1.0
2: 1.0*1_2 - 1.0*2_3 + 1.0*3_2 - 1.0*2_5 + 1.0*5_2 = 0.0
3: 1.0*1_3 + 1.0*2_3 - 1.0*3_2 - 1.0*3_5 + 1.0*5_3 = 0.0
4: 1.0*1_4 - 1.0*4_6 = 0.0
5: 1.0*2_5 - 1.0*5_2 + 1.0*3_5 - 1.0*5_3 - 1.0*5_6 + 1.0*6_5 - 1.0*5_7 = 0.0
6: 1.0*4_6 + 1.0*5_6 - 1.0*6_5 - 1.0*6_7 = 0.0
7: 1.0*5_7 + 1.0*6_7 = 1.0

Bounds:
1_2 >= 0.0
1_3 >= 0.0
1_4 >= 0.0
2_3 >= 0.0
3_2 >= 0.0
2_5 >= 0.0
5_2 >= 0.0
3_5 >= 0.0
5_3 >= 0.0
4_6 >= 0.0
5_6 >= 0.0
6_5 >= 0.0
5_7 >= 0.0
6_7 >= 0.0


In [19]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    if v.solution_value() != 0:
        print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 17.00
1_2 = 1.00
2_5 = 1.00
5_6 = 1.00
6_7 = 1.00


In [20]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
     1_2    0.0    1.0    inf   0.00
     1_3    0.0    0.0    inf   0.00
     1_4    0.0    0.0    inf   0.00
     2_3    0.0    0.0    inf   1.00
     3_2    0.0    0.0    inf   5.00
     2_5    0.0    1.0    inf   0.00
     5_2    0.0    0.0    inf  10.00
     3_5    0.0    0.0    inf   1.00
     5_3    0.0    0.0    inf   7.00
     4_6    0.0    0.0    inf   7.00
     5_6    0.0    1.0    inf   0.00
     6_5    0.0    0.0    inf   4.00
     5_7    0.0    0.0    inf   1.00
     6_7    0.0    1.0    inf   0.00


In [21]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
         1   -1.0   -1.0   -1.0  -18.00
         2    0.0    0.0    0.0  -11.00
         3    0.0    0.0    0.0  -9.00
         4    0.0    0.0    0.0   0.00
         5    0.0    0.0    0.0  -6.00
         6    0.0    0.0    0.0  -4.00
         7    1.0    1.0    1.0  -1.00
